In [101]:
import sys
sys.path.append('../../../')
from sklearn.datasets import *
from data_tools import *
from data_tools import PandasTools as pt
from data_tools import ChartTools as ct
from data_tools import AnalyzeTools as at
from data_tools import SKLTools as st
from sklearn.preprocessing import LabelEncoder
from scipy.stats import skew
ct.IPYTHON_MODE = True
import pandas as pd

train = pt.read_csv("train.csv")
train.set_index(['Id'], inplace=True)
test = pt.read_csv("test.csv")
test.set_index(['Id'], inplace=True)
target = train['SalePrice']
data = train.drop('SalePrice', axis=1).append(test)

============= Schema Info ===============
Path:train.csv
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 81 columns):
Id               1460 non-null int64
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1201 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            91 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
Utilities        1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl

In [102]:
# base feature engineering

# train['SalePrice']=train['SalePrice'].astype(float)
# train.groupby(['MasVnrType'])['SalePrice'].agg(['mean','median','count']).sort_values('mean')


# 对于未标注空值含义的分类特征填充众数
cols2 = ["MSZoning", "MasVnrType", "BsmtFullBath", "BsmtHalfBath", "Utilities", "Functional",
         "Electrical", "KitchenQual", "SaleType", "Exterior1st", "Exterior2nd"]
for col in cols2:
    data[col].fillna(data[col].mode()[0], inplace=True)
    
cols=["MasVnrArea", "BsmtUnfSF", "TotalBsmtSF", "GarageCars",
      "BsmtFinSF2", "BsmtFinSF1", "GarageArea"]
for col in cols:
    data[col].fillna(data[col].median(), inplace=True)

cols1 = ["MiscFeature", "Alley", "Fence", "GarageFinish", "GarageYrBlt", "GarageType", "MasVnrType"]
for col in cols1:
    data[col].fillna("Nan", inplace=True)
    
# 转化评价类型
rating_col = ['ExterQual', 'ExterCond', 'BsmtQual', 'BsmtCond', 'BsmtExposure',
              'BsmtFinType1', 'BsmtFinType2', 'HeatingQC', 'KitchenQual', 'FireplaceQu', 'PoolQC', 'GarageQual', 'GarageCond']
data[rating_col] = data[rating_col].fillna('Nan')   
    
rating_dic = {
    'Ex': 5, 'GLQ': 5,
    'Gd': 4, 'ALQ': 4,
    'TA': 3, 'Av': 3, 'BLQ': 3,
    'Fa': 2, 'Rec': 1,
    'Po': 1, 'Mn': 1, 'LwQ': 1,
    'No': 0, 'Unf': 0,
    'Nan': -1
}
for col in rating_col:
    data[col] = data[col].map(lambda x: rating_dic[x])



NumStr = ["MSSubClass","BsmtFullBath","BsmtHalfBath","HalfBath","BedroomAbvGr","KitchenAbvGr","MoSold","YrSold","YearBuilt","YearRemodAdd","LowQualFinSF","GarageYrBlt"]
for col in NumStr:
    data[col]=data[col].astype(str)
# 对类型进行encoding
# encoding_list = ['Street', 'Alley', 'LandContour', 'LotShape',
#                  'Utilities', 'LandSlope', 'Condition2', 'RoofStyle',
#                  'CentralAir', 'Electrical', 'MiscFeature','MoSold']
# for col in encoding_list:
#     le = LabelEncoder()
#     data[col] = le.fit_transform(data[col])

# 下述类型特征对lable均值、中位数统计后做类型分箱
data["oMSSubClass"] = data.MSSubClass.map({'180': 1,
                                          '30': 2, '45': 2,
                                          '190': 3, '50': 3, '90': 3,
                                          '85': 4, '40': 4, '160': 4,
                                          '70': 5, '20': 5, '75': 5, '80': 5, '150': 5,
                                          '120': 6, '60': 6})
data["oMSZoning"] = data.MSZoning.map(
    {'C (all)': 1, 'RH': 2, 'RM': 2, 'RL': 3, 'FV': 4})

#data["LotShape"] = data.LotShape.map({'Reg':1, 'IR1':2, 'IR2':3, 'IR3':3})
data["oLotConfig"] = data.LotConfig.map(
    {'CulDSac': 4, 'FR3': 3, 'FR2': 2, 'Inside': 2, 'Corner': 1})
#data["Condition2"] = data.Condition2.map({'PosA':1, 'PosN':2, 'FR2':3, 'Inside':3,'Corner':4})
data["oRoofMatl"] = data.RoofMatl.map({'WdShngl': 5,
                                      'WdShake': 4, 'Membran': 4,
                                      'CompShg': 3, 'ClyTile': 3,
                                      'Metal': 2, 'Tar&Grv': 2,  # ？
                                      'Roll': 1
                                      })


data["oNeighborhood"] = data.Neighborhood.map({'MeadowV': 1,
                                              'IDOTRR': 2, 'BrDale': 2,
                                              'OldTown': 3, 'Edwards': 3, 'BrkSide': 3,
                                              'Sawyer': 4, 'Blueste': 4, 'SWISU': 4, 'NAmes': 4,
                                              'NPkVill': 5, 'Mitchel': 5,
                                              'SawyerW': 6, 'Gilbert': 6, 'NWAmes': 6,
                                              'Blmngtn': 7, 'CollgCr': 7, 'ClearCr': 7, 'Crawfor': 7,
                                              'Veenker': 8, 'Somerst': 8, 'Timber': 8,
                                              'StoneBr': 9,
                                              'NoRidge': 10, 'NridgHt': 10})
data["oCondition1"] = data.Condition1.map({'Artery': 1,
                                          'Feedr': 2, 'RRAe': 2,
                                          'Norm': 3, 'RRAn': 3,
                                          'PosN': 4, 'RRNe': 4,
                                          'PosA': 5, 'RRNn': 5})

data["oBldgType"] = data.BldgType.map(
    {'2fmCon': 1, 'Duplex': 1, 'Twnhs': 1, '1Fam': 2, 'TwnhsE': 2})
data["oHouseStyle"] = data.HouseStyle.map({'1.5Unf': 1,
                                          '1.5Fin': 2, '2.5Unf': 2, 'SFoyer': 2,
                                          '1Story': 3, 'SLvl': 3,
                                          '2Story': 4, '2.5Fin': 4})
data["oExterior1st"] = data.Exterior1st.map({'BrkComm': 1,
                                            'AsphShn': 2, 'CBlock': 2, 'AsbShng': 2,
                                            'WdShing': 3, 'Wd Sdng': 3, 'MetalSd': 3, 'Stucco': 3, 'HdBoard': 3,
                                            'BrkFace': 4, 'Plywood': 4,
                                            'VinylSd': 5,
                                            'CemntBd': 6,
                                            'Stone': 7, 'ImStucc': 7})

data["oExterior2nd"] = data.Exterior2nd.map({'CBlock': 1, 'AsbShng': 1,
                                            'Brk Cmn': 2,
                                            'AsphShn': 3, 'Wd Sdng': 3, 'MetalSd': 3,
                                            'Stucco': 4,
                                            'Stone': 5,
                                            'Wd Shng': 6,
                                            'HdBoard': 7, 'Plywood': 7,
                                            'BrkFace': 8, 'VinylSd': 9, 'CmentBd': 10,
                                            'ImStucc': 11, 'Other': 12
                                            })

data["oMasVnrType"] = data.MasVnrType.map(
    {'BrkCmn': 1, 'None': 1, 'BrkFace': 2, 'Stone': 3})

data["oFoundation"] = data.Foundation.map({'Slab': 1,
                                          'BrkTil': 2, 'CBlock': 2, 'Stone': 2,
                                          'Wood': 3, 'PConc': 4})

data["oHeating"] = data.Heating.map(
    {'Floor': 1, 'Grav': 1, 'Wall': 2, 'OthW': 3, 'GasW': 4, 'GasA': 5})


data["oFunctional"] = data.Functional.map(
    {'Maj2': 1, 'Maj1': 2, 'Min1': 2, 'Min2': 2, 'Mod': 2, 'Sev': 2, 'Typ': 3})

#data["FireplaceQu"] = data.FireplaceQu.map({'None':1, 'Po':1, 'Fa':2, 'TA':3, 'Gd':4, 'Ex':5})

data["oGarageType"] = data.GarageType.map({'CarPort': 1, 'Nan': 1,
                                          'Detchd': 2,
                                          '2Types': 3, 'Basment': 3,
                                          'Attchd': 4, 'BuiltIn': 5})

data["oGarageFinish"] = data.GarageFinish.map(
    {'Nan': 1, 'Unf': 2, 'RFn': 3, 'Fin': 4})

data["oPavedDrive"] = data.PavedDrive.map({'N': 1, 'P': 2, 'Y': 3})

data["oBsmtFullBath"] = data.BsmtFullBath.map({'0.0': 1, 
                                               '3.0': 2, '1.0': 
                                               3, '2.0': 3})
data["oHalfBath"] = data.HalfBath.map({'2': 1, '0': 1, '1': 2})

data["oFence"] = data.Fence.map(
    {'MnWw': 1, 'GdWo': 2, 'MnPrv': 2, 'GdPrv': 3, 'Nan': 3})


data["oSaleType"] = data.SaleType.map({'COD': 1, 'ConLD': 1, 'ConLI': 1, 'ConLw': 1, 'Oth': 1, 'WD': 1,
                                      'CWD': 2, 'Con': 3, 'New': 3})

data["oSaleCondition"] = data.SaleCondition.map(
    {'AdjLand': 1, 'Abnorml': 2, 'Alloca': 2, 'Family': 2, 'Normal': 3, 'Partial': 4})


# LotArea等频分区后，LotFrontage在LotAread的分区间填充空值
data["LotAreaCut"] = pd.qcut(data.LotArea, 10)
data['LotFrontage'] = data.groupby(['LotAreaCut'])[
    'LotFrontage'].transform(lambda x: x.fillna(x.median()))
data.drop("LotAreaCut", axis=1, inplace=True)

# 方式1：处理年份特征
# year_cols=['YearRemodAdd','YearBuilt','YrSold','GarageYrBlt']
# data[year_cols]=data[year_cols].astype(float)
# data['YearRemodAdd']=(data['YrSold']-data['YearRemodAdd']).astype(int)
# data['YearBuilt']=(data['YrSold']-data['YearBuilt']).astype(int)
# le = LabelEncoder()
# data["GarageYrBlt"] = le.fit_transform(data["GarageYrBlt"])
# data = at.dummies(data, ['YrSold'])
# num_cols=list(data.select_dtypes(include=["float","int64"]).columns) +['YearRemodAdd','YearBuilt','GarageYrBlt']

# 方式2：处理年份特征
le = LabelEncoder()
data["GarageYrBlt"] = le.fit_transform(data["GarageYrBlt"])
data["YearBuilt"] = le.fit_transform(data["YearBuilt"])
data["YearRemodAdd"] = le.fit_transform(data["YearRemodAdd"])
data = at.dummies(data, ['YrSold'])
num_cols=list(data.select_dtypes(include=["float","int64"]).columns)
data = at.z_scores_std(data, num_cols)
data = pd.get_dummies(data)


# le = LabelEncoder()
# data["GarageYrBlt"] = le.fit_transform(data["GarageYrBlt"])
# data["YearBuilt"] = le.fit_transform(data["YearBuilt"])
# data["YearRemodAdd"] = le.fit_transform(data["YearRemodAdd"])

# X_numeric=data.select_dtypes(exclude=["object"])
# skewness = X_numeric.apply(lambda x: skew(x))
# skewness_features = skewness[abs(skewness) >= 1].index
# data = at.z_scores_std(data, list(skewness_features))
# data = pd.get_dummies(data)



In [103]:
# train_data = data[:1460]
# test_data = data[1460:]

pt.write_csv(data.reset_index(), 'data_feature_engineering.csv')